# Parte 1

### 1.1

Elegimos un dataset llamado [\"salary.csv\"](https://www.kaggle.com/datasets/ayessa/salary-prediction-classification) que tiene de utilidad original crear un algoritmo predictivo para poder predecir si una persona gana más de 50.000 USD al año. Creer que existe una desigualdad en la proporción de quien gana más dependiendo del sexo en este dataset es válido, es por esta razón que elegimos esta dataset para poder estudiar su posible sesgo.

Primero, importar los paquetes necesarios para poder utilizar AIF360 y el dataset.



In [181]:
import sys
sys.path.insert(1, "../")

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing
from IPython.display import Markdown, display
from aif360.datasets import StandardDataset

import pandas as pd
import numpy as np

np.random.seed(0)

### 1.2

Preprocesamos el dataset para poder ocupar las funciones de AIF360.

In [182]:
salary = pd.read_csv('salary.csv')

salary['sex'] = salary['sex'].apply(lambda x: 0 if x==' Female' else 1)
salary['salary'] = salary['salary'].apply(lambda x: 1 if x==' >50K' else 0)
salary_aif = StandardDataset(salary, label_name='salary', protected_attribute_names=['sex'],
                              privileged_classes=[[1]], favorable_classes=[1],
                                features_to_drop=['workclass', 'fnlwgt', 'education', 'marital-status',
                                                  'occupation', 'relationship', 'race', 'native-country'])


salary_aif_train, salary_aif_test = salary_aif.split([0.7], shuffle=True)
# salary_aif_test, salary_val_aif = salary_aif_test.split([0.7], shuffle=True)
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

In [183]:
print(salary_aif.features.shape, salary_aif_train.features.shape, salary_aif_test.features.shape)


(32561, 6) (22792, 6) (9769, 6)


Ahora el dataset puede ocupar funciones de AIF360, ya que fue procesado correctamente.

### 1.3

Seleccionamos al grupo \"Male\" como privilegiado, y \"Female\" como no privilegiado. Decidimos hacer esto ya que el número de articulos que apoya esta suposicion es bien grande. [Por ejemplo](https://www.payscale.com/gender-lifetime-earnings-gap) en este articulo establecen que los hombres ganan más que las mujeres, pero no *tanto* más cuando trabajan el mismo oficio.

### 1.4

Vamos a calcular dos métricas. La diferencia en el promedio, en donde se resta los resultados favorables para el grupo privilegiado con el grupo no privilegiado. También vamos a calcular "Disparate Impact" para poder observar la proporción de resultados favorables para el grupo no privilegiado en comparación del grupo privilegiado.



In [184]:
mean_train = BinaryLabelDatasetMetric(salary_aif_train,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("Diferencia en promedio = %f" % mean_train.mean_difference())
print("Disparidad de impacto = %f" % mean_train.disparate_impact())

Diferencia en promedio = -0.195396
Disparidad de impacto = 0.363245


Como el resultado de la diferencia en promedio (mean_difference) es negativo, indica que hay menos resultados favorables para el grupo no privilegiado, por lo que si existe un sesgo. También tenemos que la disparidad de impacto es bastante baja, por lo que el grupo privilegiado en proporción ganan más que el grupo no privilegiado, lo óptimo es que la disparidad de impacto tienda a 1.

Como el resultado de la diferencia en promedio (mean_difference) es negativo, indica que hay menos resultados favorables para el grupo no privilegiado, por lo que si existe un sesgo.

### 1.5

In [185]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

In [186]:
salary_aif_test, salary_val_aif = salary_aif_test.split([0.7], shuffle=True)

train_df, _ = salary_aif_train.convert_to_dataframe()
val_df, _ = salary_val_aif.convert_to_dataframe()
test_df, _ = salary_aif_test.convert_to_dataframe()

print(f'Train set: {train_df.shape}')
print(f'Val set: {val_df.shape}')
print(f'Test set: {test_df.shape}')

Train set: (22792, 7)
Val set: (2931, 7)
Test set: (6838, 7)


In [187]:
x_train = train_df.drop('salary', axis=1)
y_train = train_df.salary

x_val = val_df.drop('salary', axis=1)
y_val = val_df.salary

In [188]:
logistic = LogisticRegression(C=0.5, penalty='l1', solver='liblinear')
logistic.fit(x_train, y_train, sample_weight=None)

linear = LinearRegression()
linear.fit(x_train, y_train)

LinearRegression()

In [189]:
def evaluate(model, X, y_true):
    if isinstance(model, LogisticRegression):
        y_pred = model.predict_proba(X)[:, 1]
    elif isinstance(model, LinearRegression):
        y_pred = model.predict(X)

    accuracy = accuracy_score(y_true, y_pred >= 0.5)
    auc = roc_auc_score(y_true, y_pred)
    return accuracy, auc

In [190]:
print('Regresion logistica')
accuracy, auc = evaluate(logistic, x_val, y_val)
print(f'Accuracy: {accuracy}')
print(f'AUC: {auc}')

print('\nRegresion lineal')
accuracy, auc = evaluate(linear, x_val, y_val)
print(f'Accuracy: {accuracy}')
print(f'AUC: {auc}')

Regresion logistica
Accuracy: 0.8225861480723302
AUC: 0.8417734073349594

Regresion lineal
Accuracy: 0.8106448311156602
AUC: 0.8327195990400214


In [191]:
logistic_pred_df = train_df.copy()
logistic_pred_df['predict'] = logistic.predict(x_train)

linear_pred_df = train_df.copy()
linear_pred_df['predict'] = linear.predict(x_train)

In [192]:
# false positives
logistic_pred_df[logistic_pred_df['salary'] == 0][logistic_pred_df['predict'] == 1]

Boolean Series key will be reindexed to match DataFrame index.


,age,education-num,sex,capital-gain,capital-loss,hours-per-week,salary,predict
12319,29.0,13.0,1.0,0.0,0.0,75.0,0.0,1.0
6964,29.0,16.0,1.0,0.0,0.0,60.0,0.0,1.0
21840,56.0,16.0,1.0,0.0,0.0,50.0,0.0,1.0
19520,63.0,13.0,1.0,0.0,0.0,40.0,0.0,1.0
26360,51.0,13.0,1.0,0.0,0.0,50.0,0.0,1.0
...,...,...,...,...,...,...,...,...
11321,75.0,10.0,1.0,0.0,1735.0,40.0,0.0,1.0
10328,70.0,14.0,1.0,0.0,0.0,8.0,0.0,1.0
15323,29.0,14.0,1.0,0.0,0.0,60.0,0.0,1.0
32340,75.0,14.0,1.0,0.0,0.0,45.0,0.0,1.0


In [193]:
logistic_pred_df

,age,education-num,sex,capital-gain,capital-loss,hours-per-week,salary,predict
22278,27.0,10.0,0.0,0.0,0.0,44.0,0.0,0.0
8950,27.0,13.0,0.0,0.0,0.0,40.0,0.0,0.0
7838,25.0,12.0,1.0,0.0,0.0,40.0,0.0,0.0
16505,46.0,3.0,1.0,0.0,1902.0,40.0,0.0,0.0
19140,45.0,7.0,1.0,0.0,2824.0,76.0,1.0,1.0
...,...,...,...,...,...,...,...,...
19631,34.0,13.0,1.0,0.0,0.0,40.0,1.0,0.0
29694,56.0,13.0,0.0,0.0,0.0,40.0,0.0,0.0
16546,34.0,11.0,1.0,0.0,0.0,40.0,0.0,0.0
17973,25.0,9.0,1.0,0.0,0.0,40.0,0.0,0.0


In [194]:
x_train = train_df.drop('salary', axis=1)
y_train = train_df.salary

x_val = val_df.drop('salary', axis=1)
y_val = val_df.salary

In [195]:

logistic = LogisticRegression(C=0.5, penalty='l1', solver='liblinear')
logistic.fit(x_train, y_train, sample_weight=None)

linear = LinearRegression()
linear.fit(x_train, y_train)

LinearRegression()

In [196]:
def evaluate(model, X, y_true):
    if isinstance(model, LogisticRegression):
        y_pred = model.predict_proba(X)[:, 1]
    elif isinstance(model, LinearRegression):
        y_pred = model.predict(X)

    accuracy = accuracy_score(y_true, y_pred >= 0.5)
    auc = roc_auc_score(y_true, y_pred)
    return accuracy, auc

In [197]:
print('Regresion logistica')
accuracy, auc = evaluate(logistic, x_val, y_val)
print(f'Accuracy: {accuracy}')
print(f'AUC: {auc}')

print('\nRegresion lineal')
accuracy, auc = evaluate(linear, x_val, y_val)
print(f'Accuracy: {accuracy}')
print(f'AUC: {auc}')

Regresion logistica
Accuracy: 0.8229273285568065
AUC: 0.8417622782864408

Regresion lineal
Accuracy: 0.8106448311156602
AUC: 0.8327195990400214


In [198]:
logistic_aif360 = StandardDataset(logistic_pred_df, label_name='salary', protected_attribute_names=['sex'],
                              privileged_classes=[[1]], favorable_classes=[1])

linear_aif360 = StandardDataset(linear_pred_df, label_name='salary', protected_attribute_names=['sex'],
                                  privileged_classes=[[1]], favorable_classes=[1])



logistic_metrics = BinaryLabelDatasetMetric(logistic_aif360,
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)

linear_metrics = BinaryLabelDatasetMetric(linear_aif360,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=privileged_groups)

print("Diferencia en promedio regresion logistica = %f" % logistic_metrics.mean_difference())
print("Disparidad de impacto regresion logistica = %f" % logistic_metrics.disparate_impact())

print("Diferencia en promedio regresion lineal = %f" % linear_metrics.mean_difference())
print("Disparidad de impacto regresion lineal = %f" % linear_metrics.disparate_impact())

Diferencia en promedio regresion logistica = -0.195396
Disparidad de impacto regresion logistica = 0.363245
Diferencia en promedio regresion lineal = -0.195396
Disparidad de impacto regresion lineal = 0.363245


In [199]:
logistic_pred_df = train_df.copy()
logistic_pred_df['predict'] = logistic.predict(x_train)

linear_pred_df = train_df.copy()
linear_pred_df['predict'] = linear.predict(x_train)

### 1.6

In [200]:
# false positives sex == 0
fp_0 = logistic_pred_df[logistic_pred_df['salary'] == 0][logistic_pred_df['predict'] == 1][logistic_pred_df['sex'] == 0].shape[0]

# false positives sex == 1
fp_1 = logistic_pred_df[logistic_pred_df['salary'] == 0][logistic_pred_df['predict'] == 1][logistic_pred_df['sex'] == 1].shape[0]

# false negetive sex == 0
fn_0 = logistic_pred_df[logistic_pred_df['salary'] == 1][logistic_pred_df['predict'] == 0][logistic_pred_df['sex'] == 0].shape[0]

# false negetive sex == 0
fn_1 = logistic_pred_df[logistic_pred_df['salary'] == 1][logistic_pred_df['predict'] == 0][logistic_pred_df['sex'] == 1].shape[0]


# true positive sex == 0
tp_0 = logistic_pred_df[logistic_pred_df['salary'] == 1][logistic_pred_df['predict'] == 1][logistic_pred_df['sex'] == 0].shape[0]

# true positive sex == 1
tp_1 = logistic_pred_df[logistic_pred_df['salary'] == 1][logistic_pred_df['predict'] == 1][logistic_pred_df['sex'] == 1].shape[0]

n_0 = logistic_pred_df[logistic_pred_df['sex'] == 0].shape[0]
n_1 = logistic_pred_df[logistic_pred_df['sex'] == 1].shape[0]

tp_1/n_1

Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame index.


0.1437082624067042

In [201]:
logistic = LogisticRegression(C=0.5, penalty='l1', solver='liblinear')
logistic.fit(x_train, y_train, sample_weight=None)

linear = LinearRegression()
linear.fit(x_train, y_train)

LinearRegression()

In [202]:
def evaluate(model, X, y_true):
    if isinstance(model, LogisticRegression):
        y_pred = model.predict_proba(X)[:, 1]
    elif isinstance(model, LinearRegression):
        y_pred = model.predict(X)

    accuracy = accuracy_score(y_true, y_pred >= 0.5)
    auc = roc_auc_score(y_true, y_pred)
    return accuracy, auc

In [203]:
print('Regresion logistica')
accuracy, auc = evaluate(logistic, x_val, y_val)
print(f'Accuracy: {accuracy}')
print(f'AUC: {auc}')

print('\nRegresion lineal')
accuracy, auc = evaluate(linear, x_val, y_val)
print(f'Accuracy: {accuracy}')
print(f'AUC: {auc}')

Regresion logistica
Accuracy: 0.8222449675878539
AUC: 0.841712524893063

Regresion lineal
Accuracy: 0.8106448311156602
AUC: 0.8327195990400214


In [204]:
logistic_pred_df = train_df.copy()
logistic_pred_df['predict'] = logistic.predict(x_train)

linear_pred_df = train_df.copy()
linear_pred_df['predict'] = linear.predict(x_train)

In [205]:
# false positives
logistic_pred_df[logistic_pred_df['salary'] == 0][logistic_pred_df['predict'] == 1]

Boolean Series key will be reindexed to match DataFrame index.


,age,education-num,sex,capital-gain,capital-loss,hours-per-week,salary,predict
12319,29.0,13.0,1.0,0.0,0.0,75.0,0.0,1.0
6964,29.0,16.0,1.0,0.0,0.0,60.0,0.0,1.0
21840,56.0,16.0,1.0,0.0,0.0,50.0,0.0,1.0
19520,63.0,13.0,1.0,0.0,0.0,40.0,0.0,1.0
26360,51.0,13.0,1.0,0.0,0.0,50.0,0.0,1.0
...,...,...,...,...,...,...,...,...
11321,75.0,10.0,1.0,0.0,1735.0,40.0,0.0,1.0
10328,70.0,14.0,1.0,0.0,0.0,8.0,0.0,1.0
15323,29.0,14.0,1.0,0.0,0.0,60.0,0.0,1.0
32340,75.0,14.0,1.0,0.0,0.0,45.0,0.0,1.0


In [206]:
logistic_pred_df

,age,education-num,sex,capital-gain,capital-loss,hours-per-week,salary,predict
22278,27.0,10.0,0.0,0.0,0.0,44.0,0.0,0.0
8950,27.0,13.0,0.0,0.0,0.0,40.0,0.0,0.0
7838,25.0,12.0,1.0,0.0,0.0,40.0,0.0,0.0
16505,46.0,3.0,1.0,0.0,1902.0,40.0,0.0,0.0
19140,45.0,7.0,1.0,0.0,2824.0,76.0,1.0,1.0
...,...,...,...,...,...,...,...,...
19631,34.0,13.0,1.0,0.0,0.0,40.0,1.0,0.0
29694,56.0,13.0,0.0,0.0,0.0,40.0,0.0,0.0
16546,34.0,11.0,1.0,0.0,0.0,40.0,0.0,0.0
17973,25.0,9.0,1.0,0.0,0.0,40.0,0.0,0.0


In [207]:
# # logistic_aif360 = StandardDataset(logistic_pred_df, label_name='salary', protected_attribute_names=['sex'],
# #                               privileged_classes=[[1]], favorable_classes=[1])
#
# linear_aif360 = StandardDataset(linear_pred_df, label_name='salary', protected_attribute_names=['sex'],
#                                   privileged_classes=[[1]], favorable_classes=[1])
#
#
# #
# # logistic_metrics = BinaryLabelDatasetMetric(logistic_aif360,
# #                                             unprivileged_groups=unprivileged_groups,
# #                                             privileged_groups=privileged_groups)
#
# linear_metrics = BinaryLabelDatasetMetric(linear_aif360,
#                                           unprivileged_groups=unprivileged_groups,
#                                           privileged_groups=privileged_groups)
#
# # print("Diferencia en promedio regresion logistica = %f" % logistic_metrics.mean_difference())
# # print("Disparidad de impacto regresion logistica = %f" % logistic_metrics.disparate_impact())
#
# # print("Diferencia en promedio regresion lineal = %f" % linear_metrics.mean_difference())
# print("Disparidad de impacto regresion lineal = %f" % linear_metrics.disparate_impact())

Como el resultado de la diferencia en promedio (mean_difference) es negativo, indica que hay menos resultados favorables para el grupo no privilegiado, por lo que si existe un sesgo.

### 1.7

Pre-processing.

In [208]:
from aif360.algorithms.preprocessing import Reweighing

salary_aif_train, salary_aif_test = salary_aif.split([0.7], shuffle=True)
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]


pre = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
pre_train = pre.fit_transform(salary_aif_train)
pre_train_metric = BinaryLabelDatasetMetric(pre_train,
                                        unprivileged_groups=unprivileged_groups,
                                        privileged_groups=privileged_groups)

# Metricas Pre
print(pre_train_metric.mean_difference())
print(pre_train_metric.disparate_impact())

#df para 1.9
df_metricas = pd.DataFrame(data={'mean_difference': [], 'disparate_impact': []})
df2 = pd.DataFrame([[pre_train_metric.mean_difference(), pre_train_metric.disparate_impact()]], columns=['mean_difference', 'disparate_impact'])
df_metricas = pd.concat([df_metricas, df2])
df_metricas

-1.1102230246251565e-16
0.9999999999999996


,mean_difference,disparate_impact
0,-1.110223e-16,1.0


In-Processing.

In [209]:
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

salary_aif_train, salary_aif_test = salary_aif.split([0.7], shuffle=True)
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

tf.reset_default_graph()
sess = tf.Session()
adversarial= AdversarialDebiasing(privileged_groups=privileged_groups,
                                    unprivileged_groups=unprivileged_groups,
                                    scope_name='plain_classifier',
                                    debias=False,
                                    sess=sess)
adversarial.fit(salary_aif_train)
adversarial_train = adversarial.predict(salary_aif_train)
adversarial_test = adversarial.predict(salary_aif_test)
sess.close()

salary_metric_train = BinaryLabelDatasetMetric(adversarial_train,
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
salary_metric_test = BinaryLabelDatasetMetric(adversarial_test,
                                              unprivileged_groups=unprivileged_groups,
                                              privileged_groups=privileged_groups)

#In metrics
print(salary_metric_train.mean_difference())
print(salary_metric_train.disparate_impact())

print(salary_metric_test.mean_difference())
print(salary_metric_test.disparate_impact())

#df para 1.9
df3 = pd.DataFrame([[salary_metric_train.mean_difference(), salary_metric_train.disparate_impact()]], columns=['mean_difference', 'disparate_impact'])
df4 = pd.DataFrame([[salary_metric_test.mean_difference(), salary_metric_test.disparate_impact()]], columns=['mean_difference', 'disparate_impact'])
df_metricas = pd.concat([df_metricas, df3, df4])
df_metricas
adversarial

epoch 0; iter: 0; batch classifier loss: 19.136044
epoch 1; iter: 0; batch classifier loss: 10.896181
epoch 2; iter: 0; batch classifier loss: 13.357958
epoch 3; iter: 0; batch classifier loss: 2.912875
epoch 4; iter: 0; batch classifier loss: 2.735672
epoch 5; iter: 0; batch classifier loss: 2.151693
epoch 6; iter: 0; batch classifier loss: 3.599186
epoch 7; iter: 0; batch classifier loss: 3.433097
epoch 8; iter: 0; batch classifier loss: 1.331930
epoch 9; iter: 0; batch classifier loss: 1.083608
epoch 10; iter: 0; batch classifier loss: 1.447832
epoch 11; iter: 0; batch classifier loss: 1.904807
epoch 12; iter: 0; batch classifier loss: 1.467019
epoch 13; iter: 0; batch classifier loss: 0.551561
epoch 14; iter: 0; batch classifier loss: 0.702436
epoch 15; iter: 0; batch classifier loss: 0.459429
epoch 16; iter: 0; batch classifier loss: 0.476955
epoch 17; iter: 0; batch classifier loss: 0.456829
epoch 18; iter: 0; batch classifier loss: 0.383541
epoch 19; iter: 0; batch classifier lo

Post-processing.

In [210]:
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing

salary_aif_train, salary_aif_test = salary_aif.split([0.7], shuffle=True)
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]



post = CalibratedEqOddsPostprocessing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
post_train = post.fit(salary_aif_train, salary_aif_train)
pred_post = post_train.predict(salary_aif_test)
post_train_metric = BinaryLabelDatasetMetric(pred_post,
                                        unprivileged_groups=unprivileged_groups,
                                        privileged_groups=privileged_groups)

# Metricas Post
print(post_train_metric.mean_difference())
print(post_train_metric.disparate_impact())

#df para 1.9
df5 = pd.DataFrame([[post_train_metric.mean_difference(), post_train_metric.disparate_impact()]], columns=['mean_difference', 'disparate_impact'])
df_metricas = pd.concat([df_metricas, df5])
df_metricas

-0.198709076817563
0.3577001128225591


,mean_difference,disparate_impact
0,-1.110223e-16,1.000000
0,-2.013841e-01,0.207740
0,-2.058298e-01,0.212961
0,-1.987091e-01,0.357700


### 1.8
Combinacion de los algoritmos.

In [257]:
salary_aif_train, salary_aif_test = salary_aif.split([0.7], shuffle=True)
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

pre = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
pre_train = pre.fit_transform(salary_aif_train)
pre_train_metric = BinaryLabelDatasetMetric(pre_train,
                                        unprivileged_groups=unprivileged_groups,
                                        privileged_groups=privileged_groups)


tf.reset_default_graph()
sess = tf.Session()
adversarial= AdversarialDebiasing(privileged_groups=privileged_groups,
                                    unprivileged_groups=unprivileged_groups,
                                    scope_name='plain_classifier',
                                    debias=False,
                                    sess=sess)
adversarial.fit(pre_train)
adversarial_train = adversarial.predict(salary_aif_train)
adversarial_test = adversarial.predict(salary_aif_test)
sess.close()

post = CalibratedEqOddsPostprocessing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
post_train = post.fit(adversarial_train, adversarial_train)
pred_post = post_train.predict(adversarial_test)
post_train_metric = BinaryLabelDatasetMetric(pred_post,
                                        unprivileged_groups=unprivileged_groups,
                                        privileged_groups=privileged_groups)

print('Algoritmos Juntos:',post_train_metric.mean_difference())
print('Algoritmos Juntos:',post_train_metric.disparate_impact())

#df para 1.9
df6 = pd.DataFrame([[post_train_metric.mean_difference(), post_train_metric.disparate_impact()]], columns=['mean_difference', 'disparate_impact'])
df_metricas = pd.concat([df_metricas, df6])
df_metricas

epoch 0; iter: 0; batch classifier loss: 73.356247
epoch 1; iter: 0; batch classifier loss: 44.765972
epoch 2; iter: 0; batch classifier loss: 3.475793
epoch 3; iter: 0; batch classifier loss: 9.682909
epoch 4; iter: 0; batch classifier loss: 6.936759
epoch 5; iter: 0; batch classifier loss: 2.141310
epoch 6; iter: 0; batch classifier loss: 0.703779
epoch 7; iter: 0; batch classifier loss: 0.786417
epoch 8; iter: 0; batch classifier loss: 0.727921
epoch 9; iter: 0; batch classifier loss: 2.343074
epoch 10; iter: 0; batch classifier loss: 0.947923
epoch 11; iter: 0; batch classifier loss: 0.901190
epoch 12; iter: 0; batch classifier loss: 0.731843
epoch 13; iter: 0; batch classifier loss: 0.846033
epoch 14; iter: 0; batch classifier loss: 0.896007
epoch 15; iter: 0; batch classifier loss: 0.656361
epoch 16; iter: 0; batch classifier loss: 0.422647
epoch 17; iter: 0; batch classifier loss: 0.482275
epoch 18; iter: 0; batch classifier loss: 0.332469
epoch 19; iter: 0; batch classifier los

,mean_difference,disparate_impact
Pre_metricas,-1.110223e-16,1.000000
In_metricas_train,-2.013841e-01,0.207740
In_metricas_test,-2.058298e-01,0.212961
Post_metricas,-1.987091e-01,0.357700
Metricas_juntas,-2.095050e-01,0.224748
0,-1.363136e-01,0.199676
0,-1.728922e-01,0.000000


### 1.9

In [212]:
# df_metricas = df_metricas.rename(index=['Pre_metricas', 'In_metricas_train', 'In_metricas_test' 'Post_metricas', 'Metricas_juntas'])
df_metricas.index = ['Pre_metricas', 'In_metricas_train', 'In_metricas_test', 'Post_metricas', 'Metricas_juntas']
df_metricas



,mean_difference,disparate_impact
Pre_metricas,-1.110223e-16,1.000000
In_metricas_train,-2.013841e-01,0.207740
In_metricas_test,-2.058298e-01,0.212961
Post_metricas,-1.987091e-01,0.357700
Metricas_juntas,-2.095050e-01,0.224748


# Parte 2.

### 2.1

Recordemos que el objetivo de “salary.csv” es predecir quien gana más de 50.000USD. Para poder analizar esto elegimos al grupo privilegiado como los hombres, y el no privilegiado a las mujeres. En general ocupamos dos métricas. (1) mean_difference, en donde restamos los resultados favorables para el grupo privilegiado con el no privilegiado. (2) disparate_imapct, para poder analizar la proporción de resultados favorables para el grupo no privilegiado en comparación con el grupo privilegiado. 

Aplicando estas métricas al dataset original, considerando los resultados en la parte 1.4. podemos afirmar que hay menos resultados favorables para el grupo no privilegiado (mujeres), y que el grupo privilegiado (hombres) en general gana más que el grupo no privilegiado. Aunque la diferencia es clara, uno podría suponer que la brecha era más grande, y talvez así lo sea, pero el dataset no lo representa. 

Aplicando el algoritmo de pre-processing “Reweighing”, tenemos que la mean_difference es mayor que en el dataset original, mientras que el disparate_impact está cercano a su valor optimo, el cual es 1, por lo que en proporción gracias a este algoritmo ambos ganan cantidades similares.  

En in-processing, utilizamos un algoritmo llamado “AdversarialDebiasing”. Observamos que disparate_impact bajo en comparación con el dataset original, y la mean_difference se acentuó, por lo que por sí solo no es un buen cambio. 

En post-processing utilizamos “CalibratedEqOddsPostprocessing”. Aquí la mean_difference sigue siendo negativa, y la disparate_impact no está muy cercana a 1, por lo que por sí solo tampoco es un cambio muy útil. 

Todos los algoritmos juntos tampoco son muy optimistas los cambios, por lo que podemos afirmar con seguridad que ó: (1) No estamos ocupando los algoritmos correctos, ó (2) el dataset ya se encuentra relativamente bien balanceado. En nuestra opinión, esto es mitad verdad, mitad falso. En el dataset original mean_difference era de -0.19, lo cual no es óptimo, pero tampoco terriblemente malo. El disparate_imapct era de 0.36, claramente podría ser mejor y es aquí en donde tal vez mejorar la elección de algoritmos lo haya solucionado.

### 2.2

Podemos sacar 3 conclusiones claras:

1.	Dataset original definitivamente tiene sesgo hacia un grupo privilegiado, no es perfecta la distribución, pero claramente podría ser peor. 

2.	Algoritmos que ocupamos no fueron efectivos en balancear el sesgo. 

3.	Las dos principales métricas que ocupamos pudieron efectivamente demostrar los sesgos presentes en el dataset.

### 2.3

Como solamente pudimos una vez corregir el disparate_impact, el algoritmo de AIF360 llamado “Disparate Impact Remover” podría haber sido útil. Este algoritmo es de tipo pre-processing, se encarga de editar valores de crecimiento para fairness de grupo mientras que preserva el orden dentro de esos mismos grupos.

### 2.4

### 2.5

In [274]:
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

from aif360.datasets import StandardDataset
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing


class Fairness(StandardDataset):
    def __init__(self, df:pd.DataFrame, label_name:str, protected_attribute_names:list, privileged_groups:list, unprivileged_groups:list, *args, **kwargs):

        self.__orginal_df = df
        self.label_name = label_name
        self.protected_attribute_names = protected_attribute_names
        self.privileged_groups = privileged_groups
        self.unprivileged_groups = unprivileged_groups

        # Paso de variable categorica a Numerica

        for i in protected_attribute_names:
            if df[i].dtype == 'O':
                dummie = pd.get_dummies(df[i])
                df[i] = dummie[dummie.columns[0]]

        if df[label_name].dtype == 'O':
            dummie = pd.get_dummies(df[label_name])
            df[label_name] = dummie[dummie.columns[0]]

        self.df = df

        super().__init__(self.df, label_name=self.label_name, protected_attribute_names=self.protected_attribute_names,*args, **kwargs)


    @staticmethod
    def metrics(df):
        mean_train = BinaryLabelDatasetMetric(df, unprivileged_groups = df.unprivileged_groups,
                                              privileged_groups = df.privileged_groups)

        print("Diferencia en promedio = %f" % mean_train.mean_difference())
        print("Disparidad de impacto = %f" % mean_train.disparate_impact())


    @staticmethod
    def pre_processing(df):
        pre = Reweighing(unprivileged_groups=df.unprivileged_groups,
                         privileged_groups=df.privileged_groups)

        return pre.fit_transform(df)


    @staticmethod
    def in_procesing(df):
        tf.reset_default_graph()
        sess = tf.Session()
        adversarial = AdversarialDebiasing(privileged_groups=df.privileged_groups,
                                          unprivileged_groups=df.unprivileged_groups,
                                          scope_name='plain_classifier',
                                          debias=False,
                                          sess=sess)
        adversarial.fit(df)
        adversarial_df = adversarial.predict(df)
        sess.close()
        return adversarial_df

    @staticmethod
    def pos_procesing(df):
        post = CalibratedEqOddsPostprocessing(unprivileged_groups=df.unprivileged_groups,
                                              privileged_groups=df.privileged_groups)
        df_post = post.fit(df, df)
        return  df_post.predict(df)

    @staticmethod
    def full_procesing(df):
        pre = Fairness.pre_processing(df)
        in_ = Fairness.in_procesing(df)
        # post = Fairness.pos_procesing(df)
        return in_


In [272]:
kwargs = {'label_name':'salary', 'protected_attribute_names':['sex'], 'privileged_classes':[[1]], 'favorable_classes':[1],
          'features_to_drop':['workclass', 'fnlwgt', 'education', 'marital-status','occupation', 'relationship', 'race', 'native-country'],
          'privileged_groups':[{'sex': 1}], 'unprivileged_groups':[{'sex': 0}]
          }

aa = Fairness(pd.read_csv('salary.csv'), **kwargs)

In [276]:
Fairness.metrics(Fairness.full_procesing(aa))

epoch 0; iter: 0; batch classifier loss: 47.942768
epoch 0; iter: 200; batch classifier loss: 20.199440
epoch 1; iter: 0; batch classifier loss: 10.506297
epoch 1; iter: 200; batch classifier loss: 3.194176
epoch 2; iter: 0; batch classifier loss: 4.490228
epoch 2; iter: 200; batch classifier loss: 2.772004
epoch 3; iter: 0; batch classifier loss: 4.915930
epoch 3; iter: 200; batch classifier loss: 1.765794
epoch 4; iter: 0; batch classifier loss: 3.092273
epoch 4; iter: 200; batch classifier loss: 19.727531
epoch 5; iter: 0; batch classifier loss: 1.762059
epoch 5; iter: 200; batch classifier loss: 1.718424
epoch 6; iter: 0; batch classifier loss: 0.759780
epoch 6; iter: 200; batch classifier loss: 2.005347
epoch 7; iter: 0; batch classifier loss: 1.988709
epoch 7; iter: 200; batch classifier loss: 1.321532
epoch 8; iter: 0; batch classifier loss: 0.646514
epoch 8; iter: 200; batch classifier loss: 0.399388
epoch 9; iter: 0; batch classifier loss: 0.906241
epoch 9; iter: 200; batch cl